# 機械学習フロー
# 【問題1】クロスバリデーション
事前学習期間は検証データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。


具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。
## データの確認

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

## 【問題3】Kernelからの調査
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。  
全パラメータを用いたランダムフォレストが気に入ったのでそのモデルを使用する。

## 【問題4】高い汎化性能のモデル
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。

In [2]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
app_train_notarget = app_train.drop('TARGET', axis=1) # TARGETを落としたもの
target = app_train['TARGET']
test_id = app_test[['SK_ID_CURR']].copy()

In [3]:
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
# 共通の列にしている（今回は変化なし）

In [4]:
app_train_notarget.shape, target.shape, app_test.shape

((307511, 121), (307511,), (48744, 121))

In [5]:
# どちらがテストデータかわかるようにしている(TARGETを落としたため)
app_train['training_set'] = True 
app_test['training_set'] = False

In [6]:
app_full = pd.concat([app_train, app_test], axis=0).copy() # 縦方向に結合
app_full = app_full.drop('SK_ID_CURR', axis=1)# IDは特徴量ではないので除外
app_full.shape

(356255, 121)

くっつけることによってダミーデータ作成や欠損値作成を一括で行うことができる。

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
app_full.dtypes.value_counts()

float64    65
int64      39
object     16
bool        1
dtype: int64

In [8]:
# 2値のオブジェクトをダミー変数化
le_count = 0

for col in app_full:
    if app_full[col].dtype == 'object':
         if len(list(app_full[col].unique())) <= 2: # 2個以上でそれやると何が何だかわからなくなるため２個まで
            le.fit(app_full[col])
            app_full[col] = le.transform(app_full[col])
            
            le_count += 1
print('%d columns were labl encoded.' % le_count) 

3 columns were labl encoded.


In [9]:
app_full = pd.get_dummies(app_full)

In [10]:
app_full.shape

(356255, 242)

TARGET と　 FLAG系のデータの相関がないことを確認して削除(略)

In [11]:
app_train['TARGET'] = target
app_doc_corr = app_train[['TARGET','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4',
                        'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8', 
                        'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 
                        'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16',
                        'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
                        'FLAG_DOCUMENT_21']].copy()

In [12]:
corr = app_doc_corr.corr()['TARGET'].sort_values()
corr

FLAG_DOCUMENT_6    -0.028602
FLAG_DOCUMENT_16   -0.011615
FLAG_DOCUMENT_13   -0.011583
FLAG_DOCUMENT_14   -0.009464
FLAG_DOCUMENT_8    -0.008040
FLAG_DOCUMENT_18   -0.007952
FLAG_DOCUMENT_15   -0.006536
FLAG_DOCUMENT_9    -0.004352
FLAG_DOCUMENT_11   -0.004229
FLAG_DOCUMENT_17   -0.003378
FLAG_DOCUMENT_4    -0.002672
FLAG_DOCUMENT_7    -0.001520
FLAG_DOCUMENT_10   -0.001414
FLAG_DOCUMENT_19   -0.001358
FLAG_DOCUMENT_12   -0.000756
FLAG_DOCUMENT_5    -0.000316
FLAG_DOCUMENT_20    0.000215
FLAG_DOCUMENT_21    0.003709
FLAG_DOCUMENT_2     0.005417
FLAG_DOCUMENT_3     0.044346
TARGET              1.000000
Name: TARGET, dtype: float64

In [13]:
corr.abs().sort_values(ascending=False)

TARGET              1.000000
FLAG_DOCUMENT_3     0.044346
FLAG_DOCUMENT_6     0.028602
FLAG_DOCUMENT_16    0.011615
FLAG_DOCUMENT_13    0.011583
FLAG_DOCUMENT_14    0.009464
FLAG_DOCUMENT_8     0.008040
FLAG_DOCUMENT_18    0.007952
FLAG_DOCUMENT_15    0.006536
FLAG_DOCUMENT_2     0.005417
FLAG_DOCUMENT_9     0.004352
FLAG_DOCUMENT_11    0.004229
FLAG_DOCUMENT_21    0.003709
FLAG_DOCUMENT_17    0.003378
FLAG_DOCUMENT_4     0.002672
FLAG_DOCUMENT_7     0.001520
FLAG_DOCUMENT_10    0.001414
FLAG_DOCUMENT_19    0.001358
FLAG_DOCUMENT_12    0.000756
FLAG_DOCUMENT_5     0.000316
FLAG_DOCUMENT_20    0.000215
Name: TARGET, dtype: float64

相関が高かった  
- FLAG_DOCUMENT_3     0.044346
- FLAG_DOCUMENT_6     0.028602
- FLAG_DOCUMENT_16    0.011615
- FLAG_DOCUMENT_13    0.011583  
以外のものを削除  
なんでわざわざこの特徴量だけ削除しているのかはよくわからなかった。

In [14]:
app_full = app_full.drop(['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 
                        'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 
                        'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 
                        'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15'], axis=1)

In [15]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis = 1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0:"Missing Values", 1:"% of Total Values"})
    
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending = False).round(1)
    
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
    
    return mis_val_table_ren_columns

In [16]:
app_full_miss = missing_values_table(app_full)
app_full_miss.head(20)

Your selected dataframe has 231 columns.
There are 61 columns that have missing values.


,Missing Values,% of Total Values
COMMONAREA_MODE,248360,69.7
COMMONAREA_MEDI,248360,69.7
COMMONAREA_AVG,248360,69.7
NONLIVINGAPARTMENTS_MODE,246861,69.3
NONLIVINGAPARTMENTS_AVG,246861,69.3
NONLIVINGAPARTMENTS_MEDI,246861,69.3
LIVINGAPARTMENTS_MEDI,242979,68.2
LIVINGAPARTMENTS_AVG,242979,68.2
LIVINGAPARTMENTS_MODE,242979,68.2
FLOORSMIN_MEDI,241108,67.7


欠損値補完の方法として、最頻値補完を採用している。  
70%もないデータがあるにもかかわらず無理矢理特攻していくのは驚きだ  
また、標準化ではなく正規化をしている。

In [17]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median') #最頻値で補完
scaler = MinMaxScaler(feature_range=(0, 1)) #最大値1　 最小値0に設定

In [18]:
for col in app_full.columns:
    app_full[[col]] = imputer.fit_transform(app_full[[col]])
    app_full[[col]] = scaler.fit_transform(app_full[[col]])

In [19]:
app_full.isnull().sum().sort_values(ascending=False)

NAME_CONTRACT_TYPE                      0
OCCUPATION_TYPE_Realty agents           0
OCCUPATION_TYPE_Secretaries             0
OCCUPATION_TYPE_Security staff          0
OCCUPATION_TYPE_Waiters/barmen staff    0
                                       ..
FLAG_DOCUMENT_3                         0
FLAG_DOCUMENT_6                         0
FLAG_DOCUMENT_13                        0
FLAG_DOCUMENT_16                        0
EMERGENCYSTATE_MODE_Yes                 0
Length: 231, dtype: int64

In [20]:
app_train = app_full[app_full['training_set']==1.0]

In [21]:
app_test = app_full[app_full['training_set']==0.0]

In [22]:
app_train = app_train.drop('training_set', axis=1)
app_test = app_test.drop('training_set', axis=1)

In [23]:
app_train.shape

(307511, 230)

In [24]:
app_test.shape

(48744, 230)

In [25]:
target.shape

(307511,)

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(app_train, target)

RandomForestClassifier()

In [27]:
result_rf_model = pd.DataFrame(rf_model.predict_proba(app_test)[:, 1], columns=['TARGET']) # dfにして保存
result_rf = pd.concat([test_id, result_rf_model], axis=1)
RESULT_rf = result_rf.set_index('SK_ID_CURR') # 結合とindexの振り直し
RESULT_rf.to_csv('result rf.csv')

In [28]:
RESULT_rf

,TARGET
SK_ID_CURR,
100001,0.10
100005,0.14
100013,0.03
100028,0.04
100038,0.15
...,...
456221,0.04
456222,0.04
456223,0.06


## ランダムサーチ

In [29]:
# from sklearn.model_selection import RandomizedSearchCV
# n_estimator = np.arange(50, 300, 50)
# min_samples_split = np.arange(2, 20, 2)
# min_samples_leaf = np.arange(2, 20, 2)
# param = {'n_estimators' : n_estimator, 'min_samples_split' : min_samples_split, 'min_samples_leaf' : min_samples_leaf}
# rf_model_ranserch = RandomForestClassifier(random_state=0, n_jobs=-1)
# ran_rf = RandomizedSearchCV(rf_model_ranserch, param, cv=5, scoring='roc_auc', n_iter=20)
# ran_rf.fit(app_train, target)

In [30]:
# print('best_score : {}'.format(ran_rf.best_score_)) # このスコアは交差検証の平均値？？？
# print('best_param : {}'.format(ran_rf.best_params_))# 一番良かったのを返す

In [31]:
# ran_rf = pd.DataFrame.from_dict(ran_rf.cv_results_, orient='index').T # 結果をdataframeにして表示
# ran_rf.sort_values('rank_test_score').head(30)

In [32]:
# 一番良かった条件で学習
rf_model_fin = RandomForestClassifier(n_estimators=250, min_samples_split=16, min_samples_leaf=18, random_state=0, n_jobs=-1)

In [33]:
rf_model_fin.fit(app_train.values, target.values)

RandomForestClassifier(min_samples_leaf=18, min_samples_split=16,
                       n_estimators=250, n_jobs=-1, random_state=0)

In [34]:
rf_model_fin_pred = rf_model_fin.predict_proba(app_test.values)[:, 1]

rf_fin_res = pd.DataFrame(rf_model_fin_pred, columns=['TARGET'])
rf_fin_res_2 = pd.concat([test_id, rf_fin_res], axis=1)
rf_fin_res_3 = rf_fin_res_2.set_index('SK_ID_CURR') # 結合とindexの振り直し
rf_fin_res_3.to_csv('random forest random search.csv')